<a href="https://colab.research.google.com/github/CanteroAlann/machine-learning-projects/blob/main/TrabajoPractico1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
business_df = pd.read_csv('/content/drive/MyDrive/tp1_orga/business.csv',engine='python',on_bad_lines='skip')
business_df.dropna(inplace=True)

P1. ¿Cuál es el estado con mejor rating promedio en sus negocios? ¿Por qué?

In [2]:
# agrupo por estado sacando el promedio del puntaje 
business_by_state = business_df.groupby(['state']).agg({'stars' :['mean','count'] })
# reordeno el dataframe 
level0=business_by_state.columns.get_level_values(0)
level1=business_by_state.columns.get_level_values(1)
business_by_state.columns = level0 + '_' + level1
business_by_state.reset_index(inplace=True)

In [3]:
# voy a tomar aquellos estados que tengan mas de 2000 reviews para obtener una mejor metrica de lo pedido por el enunciado 
business_by_state = business_by_state.loc[business_by_state['stars_count'] > 2000]
# ordeno el dataframe por el promedio de cada estado 
business_by_state.sort_values(by=['stars_mean'],ascending=False,inplace=True)
business_by_state.reset_index(inplace=True,drop=True)
business_by_state

,state,stars_mean,stars_count
0,CA,3.996407,3618
1,NV,3.777778,5499
2,ID,3.754511,3436
3,LA,3.728558,7462
4,FL,3.655014,20353
5,AZ,3.629227,7452
6,PA,3.628601,25583
7,TN,3.621645,9351
8,IN,3.612330,8702
9,MO,3.586621,8693


In [4]:
# Luego el estado con mejor revies en sus negocios es 
Estado_con_mejor_promedio_en_sus_negocios = business_by_state.loc[0] 
Estado_con_mejor_promedio_en_sus_negocios

state                CA
stars_mean     3.996407
stars_count        3618
Name: 0, dtype: object

P2 . ¿Cuál es el usuario más quejumbroso por estado? Consideramos quejumbroso a un usuario que tiene más de 5 reviews y son todas de 2 estrellas o menos.

In [5]:
reviews_df = pd.read_csv('/content/drive/MyDrive/tp1_orga/review-002.csv',engine='python',on_bad_lines='skip')
reviews_df.dropna(inplace=True)
user_df = pd.read_csv('/content/drive/MyDrive/tp1_orga/user-001.csv')
user_df.dropna(inplace=True)

In [6]:
# Filtro los usuarios que tienen mas de 5 reviews 
user_df = user_df.loc[user_df['review_count'] > 4]

In [7]:
# Filtro usando los user_id del dataframe anterior y usando aquellas reviews que sean de 2 estrellas o menos
list_of_users = user_df['user_id'].to_list()
mask_1 = reviews_df['user_id'].isin(list_of_users) 
mask_2  = reviews_df['stars'] < 3
reviews_df = reviews_df[mask_1 & mask_2]

In [8]:
# agrupo por usuario y su review
user_group = reviews_df.groupby(['user_id','business_id']).size()
user_group = user_group.to_frame(name='cant')
user_group.reset_index(inplace=True)
user_group

,user_id,business_id,cant
0,--17Db1K-KujRuN7hY9Z0Q,KC8_Rx4Orlsz8LIonCYXsA,1
1,--_H9j6ggxvqhh9nPofZwg,5vFaHSP7C0gL-kpXi9SQuw,1
2,--_H9j6ggxvqhh9nPofZwg,L5Hdck6IhHmryDdRRv8U_g,1
3,--_r6E98SNIrGU7weyNxbw,1Efjww8n7WtKtKTgDqjgdA,1
4,--_r6E98SNIrGU7weyNxbw,In6HBKB32pdiOszyUsASBg,1
...,...,...,...
68833,zyyUMi-GJmXMRv-C3LCArA,VBWUdluhRQf5kqawykM6Nw,1
68834,zyyUMi-GJmXMRv-C3LCArA,oBNrLz4EDhiscSlbOl8uAw,1
68835,zzhO1jW4skCDWNrWtSLbTw,AdJh_RNF4DpeVWcdy9FDiA,1
68836,zziWJMYwDjyVi7kJmgRUvg,LjXS6sgzY6-kX2WKPykx_A,1


In [9]:
# creo una lista de negocios para filtrar el dataframe de negocios 
list_of_business = user_group['business_id'].to_list()

In [10]:
# puede que haya negocios que tengan review y no esten en el dataframe de negocios , por lo tanto vuelvo a filtrar
business_df = business_df.loc[business_df['business_id'].isin(list_of_business)]
list_of_business = business_df['business_id'].to_list()
user_group= user_group.loc[user_group['business_id'].isin(list_of_business)]

In [11]:
def func_state(x):
     return business_df.loc[x ,'state']

business_df.set_index('business_id',inplace=True)
user_group['state'] = user_group['business_id'].transform(lambda x : func_state(x))

<ipython-input-11-c51a728b985b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_group['state'] = user_group['business_id'].transform(lambda x : func_state(x))


In [12]:
# agrupo por user_id sumando la cantidad de reviews del usuario y quedandome con el estado donde mas hizo review el usuario en cuestion 
list_user = user_group.groupby(['user_id']).agg({'cant':['sum'],'state':['max']})
level0=list_user.columns.get_level_values(0)
level1=list_user.columns.get_level_values(1)
list_user.columns = level0 + '_' + level1
list_user.reset_index(inplace=True)

In [13]:
# ordeno el dataframe segun cantidad de reviews 
list_user.sort_values(by='cant_sum',ascending=False,inplace=True)
# el paso anterior me sirvio para de esta manera obtener el usuario con mas reviews por estado , si elimino los duplicados quedandome solo con la primera aparicion obtengo los usuarios mas quejumbrosos por estado 
list_user.drop_duplicates(subset='state_max',keep='first',inplace=True)
# usuarios mas quejumbrosos por estado 
list_user

,user_id,cant_sum,state_max
11329,gv0coNUFY-fibRwP8IKqPA,123,TN
12500,lSA0wc9EnIndn5F3Q6RNmw,112,NV
7521,Sp2GV7D-_JLZMPQmDanzPQ,103,FL
4931,IPcgVWB-KXlb5bqt8fqE0Q,88,AZ
11990,jK_SY5ciS1IL82Cf3Ed1_w,86,PA
2187,7Ph4w4BvyXhFYWB0bLZCFQ,58,IN
6734,PnwOegp7RXfMeNAyO9fQhQ,58,MO
8480,WjsZSsBbUTwkGUIkv7TpjQ,57,ID
3869,E4BsVQnG5zetbwv2x8QIWg,46,LA
8984,Yi0eUCpnNHSZnc0mJgfTbw,36,IL


P3.Queremos recomendarle lugares al usuario ‘bNnBwW5kNO77KTgMeVhxKg’. Para esto vamos a decir que a un usuario le gusto un lugar si le dio 3 estrellas o más y tomamos sólo los usuarios que hayan realizado más de 10 reviews. Vamos a buscar al usuario que más lugares le gusten en común y recomendarle todos aquellos que a ese le gustaran y le falte hacer review.

In [14]:
import numpy as np
review_df=pd.read_csv('/content/drive/MyDrive/tp1_orga/review-002.csv',usecols=['user_id','business_id','text','stars'])

In [15]:
# Filtro aquellas reviews con 3 estrellas o mas
aux_df = review_df['stars'] > 2 
review_df = review_df[aux_df]
# Filtro las reviews donde el ususario del enunciado hizo review 
usuario_enunciado = 'bNnBwW5kNO77KTgMeVhxKg'
aux_df = review_df['user_id'].isin([usuario_enunciado])
likedPlaces_df = review_df[aux_df]
likedPlaces_df
# voy a necesitar esta lista de lugares para filtrar mas adelante 
liked_places = likedPlaces_df['business_id'].to_list()

In [16]:
user_df=pd.read_csv('/content/drive/MyDrive/tp1_orga/user-001.csv',usecols=['user_id','name','review_count'])
# Filtro los ususarios con mas de 10 reviews
aux_df=user_df['review_count'] > 9
user_df = user_df[aux_df]
user_df

,user_id,name,review_count
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,64.0
23,MGPQVLsODMm9ZtYQW-g_OA,Jelena,12.0
29,4ZaqBJqt7laPPs8xfWvr6A,Nina,11.0
32,NIhcRW6DWvk1JQhDhXwgOQ,Lia,29.0
36,rppTTi-kfF8-qyiArNemag,Helen,25.0
...,...,...,...
1403454,CIFrgKgUrr6xyigd1AHeUw,Brent,10.0
1470095,sxwRyK91SyoN76jXwmLwdA,Brenda,10.0
1483071,1w5Fukf-OFzWUivrRJEtIA,Tom,10.0
1490033,fEpGooV1MPAW0IJW3Er-9Q,Amanda,10.0


In [17]:
# Uso los usuarios obtenidos del paso anterior para obtener los lugares donde hicieron review 
list_of_users = user_df['user_id'].to_list()
len(list_of_users)
mask=review_df['user_id'].isin(list_of_users)
review_df=review_df[mask]
# agrupo por usuario y negocio al que hizo review
reviews_for_users=review_df.groupby(['user_id','business_id']).size().to_frame(name='cant_of_reviews')
reviews_for_users.reset_index(inplace=True)
reviews_for_users

,user_id,business_id,cant_of_reviews
0,--B4MfqBxNuXX8ujyh8VXg,0AfjyySl4Rivh-QhWfrClA,1
1,--B4MfqBxNuXX8ujyh8VXg,4A_1kNLFVYmQ-zfz93huBg,1
2,--B4MfqBxNuXX8ujyh8VXg,7TkgWz7fLbMBjkS_RTTOIg,1
3,--B4MfqBxNuXX8ujyh8VXg,BEX2x7s6wH3bOTw_xrObsA,1
4,--B4MfqBxNuXX8ujyh8VXg,DKdavX6EstepxVF1OqETOw,1
...,...,...,...
1134895,zzw0Z6-_VDp9ShIRSKIsQw,gpTC5qka3HCQqnSyXlophg,1
1134896,zzw0Z6-_VDp9ShIRSKIsQw,kpRRCAN8PQTeWHGBoxpl_A,1
1134897,zzw0Z6-_VDp9ShIRSKIsQw,r5NnhGWX0qjrWBvH-65gOA,1
1134898,zzw0Z6-_VDp9ShIRSKIsQw,ruE_DRE6fTIE6who59hdfQ,2


In [18]:
# uso la lista de negocios del usuario del enunciado para asi obtener los usuarios tienen en comun al menos un negocio 
mask = reviews_for_users['business_id'].isin(liked_places)
aux_df = reviews_for_users[mask]
# el paso anterior me da los negocios que tienen en comun , ahora debo filtrar usando la lista de usuarios para asi obtener aquellos lugares que no tienen en comun
users_whit_one_place_or_more = aux_df['user_id'].to_list()
# Con este ultimo filtrado ya obtengo los usuarios y sus respectivos negocios tanto como los que tienen en comun y los que no 
mask = reviews_for_users['user_id'].isin(users_whit_one_place_or_more)
definitive_users = reviews_for_users[mask]
definitive_users

,user_id,business_id,cant_of_reviews
166,--aVoR4DrnGoia8OTfS9rQ,-9NmUeTphyS9Lq1o9MACGw,1
167,--aVoR4DrnGoia8OTfS9rQ,1A7BfewtAi27Tln39Tlgfw,1
168,--aVoR4DrnGoia8OTfS9rQ,6TUOsIU1vNNQcoauKrZFCw,1
169,--aVoR4DrnGoia8OTfS9rQ,DPtdBtLp6ng9cAB9_rzB6g,1
170,--aVoR4DrnGoia8OTfS9rQ,FeCkEyoGHVAR7KzrEX-uMA,1
...,...,...,...
1134528,zyfQkC4UITOpJNC1mLzm_A,gmYMOcIFsfUnlP_5HMaHpw,1
1134529,zyfQkC4UITOpJNC1mLzm_A,oM9ZFIgO88Hu0ppDZPjuwg,1
1134530,zyfQkC4UITOpJNC1mLzm_A,pVwMHUYFMuwmRe6M--ZzwA,2
1134531,zyfQkC4UITOpJNC1mLzm_A,v17tgCRjuPQiFOq4uvEI-w,1


In [19]:
# seteo todas las cantidades de reviews a 1 para usar esto en el siguiente paso 
definitive_users.loc[definitive_users['cant_of_reviews']  > 1, 'cant_of_reviews' ] = 1
# Armo una tabla para usar el concepto de BOW y la similidaridad coseno para de esa forma obtener el usuario que mas lugares tiene en comun con el usuario del enunciado 
table_of_users=definitive_users.set_index(['user_id', 'business_id'])['cant_of_reviews'].unstack('business_id').fillna(0)
table_of_users

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


business_id,---kPU91CF4Lq2-WlRu9Lw,--hF_3v1JmU9nlu4zfXJ8Q,--lqIzK-ZVTtgwiQM63XgQ,--sXnWH9Xm6_NvIjyuA99w,-0TffRSXXIlBYVbb5AwfTg,-0Ym1Wg3bXd_TDz8JtvOQg,-0__F9fnKt8uioCKztF5Ww,-0eUa8TsXFFy0FCxHYmrjg,-0epFLgYq2C1Jo_W4FOBKw,-0gRYq5UjMtZbELj0KHxzA,...,zwXT4m9svdg-xh2cKjEp-Q,zwfN21b6gN2XGjNG6MYNzQ,zwiiT0Nz1UM7PEXyvjZY2Q,zwrgCMuZyFX46mL3piDyjg,zxIF-bnaJ-eKIsznB7yu7A,zxuVnNVAUm16Sro0dS_lmg,zyFn7oqokhaUaPPqd2FbbA,zyPz0xvE5XkeCVH2J7g2gw,zymvTQ12tRy3bzmP4tPgOg,zzXDi0Pdv0s84M-oQaIa_g
user_id,,,,,,,,,,,,,,,,,,,,,
--aVoR4DrnGoia8OTfS9rQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--jXCQWcI7_u6f9NN7cEGA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-0KMZFU7kI1D9iF73lExqw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-0uKY1pQZBRwfiAWO86S0w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-2jOd0hhARZvhy0OTB544A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuI0ZdxyUSoxCL3W_16g-w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zwXmvn1op5LuFF2Kveqaug,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zxMPPtPPMeuX7KE29Z_vaw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# En la tabla armada se va a encontrar el usuario del enunciado , por lo tanto guardo la serie del usuario del enunciado en una variable y lo borro de la tabla 
user_target = table_of_users.loc[usuario_enunciado]
table_of_users.drop([usuario_enunciado],inplace=True)
# En esta parte usare la similaridad coseno para obtener el usuario con mas lugares en comun 
# primero obtengo una lista de los id de los usuarios 
aux=table_of_users.reset_index()
users_list= aux['user_id'].to_list()
# Porcion de codigo que se encargara de buscar al usuario con mas lugares en comun con el usuario del enunciado 
cos_definitive = 0
user_name = ''
norm_user_target = np.sqrt(user_target.dot(user_target))
for user in users_list:
  vector_user = table_of_users.loc[user]
  vector_user
  norm_vector = np.sqrt(vector_user.dot(vector_user))
  similarity_cos =  (vector_user.dot(user_target)) / (norm_vector * norm_user_target)
  if cos_definitive < similarity_cos:
    cos_definitive = similarity_cos
    user_name = user



In [21]:
# filtros para obtener la lista de negocios que le falta hacer review al usuario del enunciado 
mask = review_df['user_id'] == user_name
aux_df = review_df[mask]
mask_2 = ~aux_df['business_id'].isin(liked_places)
places_whitout_review_df = aux_df[mask_2]
places_whitout_review_df


,user_id,business_id,stars,text
11014,I-JKKX7PHHd_z9nNMkH6kg,E-4t5Hoon6aVFTWDPz26fQ,4.0,I haven't been back to Biscuits since they exp...
23365,I-JKKX7PHHd_z9nNMkH6kg,ymoEdJB5NE56sWw74uyX7A,4.0,This place is a gem! My first time here was in...
42947,I-JKKX7PHHd_z9nNMkH6kg,yeHLiKNp0hyR-ig4M6us-w,5.0,This place is delicious! First time here and h...
55140,I-JKKX7PHHd_z9nNMkH6kg,Dtuf4vD0znzwJW9vk3iTNw,4.0,The people that work where here are super nice...
59664,I-JKKX7PHHd_z9nNMkH6kg,ELKgARoj5UNjQXUkX8T8KQ,4.0,First time here and definitely will be back! I...
...,...,...,...,...
3692170,I-JKKX7PHHd_z9nNMkH6kg,8Pw3gUxSISkGqWNOsdWQ4Q,5.0,Thai Papaya is one of the best! There are seve...
3709302,I-JKKX7PHHd_z9nNMkH6kg,3GZ93AuxWsKwhhHEeTSfPw,4.0,Such a quaint neighborhood bar and grill! My b...
3725357,I-JKKX7PHHd_z9nNMkH6kg,SYffIaST3GI1FNXk9A_uwg,4.0,The best Upland by far!!! The other ones are a...
3811578,I-JKKX7PHHd_z9nNMkH6kg,zfqfw3FizcBMfU1vJvLWMQ,4.0,This is a pretty neat place that is tucked awa...


In [22]:
list_of_places_whitout_review = places_whitout_review_df['business_id'].to_list()
# Cargo el otro dataset para obtener la lista de nombres de los negocios 
business_df = pd.read_csv('/content/drive/MyDrive/tp1_orga/business.csv',engine='python',on_bad_lines='skip',usecols=['business_id','name'])
mask = business_df['business_id'].isin(list_of_places_whitout_review)
business_df = business_df[mask]
list_of_names = business_df['name'].to_list()
# respuesta al enunciado 
list_of_names

['Barnes & Noble Booksellers',
 'Bier Brewery and Tap Room',
 'Tomo Japanese Steakhouse and Sushi Bar',
 'Kroger',
 'Laundromat',
 'Sushi Bar',
 'LouVino',
 'Daredevil Hall',
 'Upland Carmel Tap House',
 'The Cuban Sandwich Shop',
 "Yong's Alterations",
 'Chatham Tap Fishers',
 'Biscuits Cafe',
 'Livery - Indianapolis',
 'JW Marriott Indianapolis',
 'Chiang Mai Thai Noodle',
 'I Love Salad',
 'Oakleys Bistro',
 'Clean Juice',
 'Old Gold Barbecue',
 'Indy Tacos',
 'One Trick Pony',
 'Margaritaville - Nashville',
 "Culver's",
 'Broad Ripple Brewpub',
 'Nail Bar',
 'Fancy Fortune Cookies',
 'Delicia',
 "Cynthia's Hallmark",
 'Staples',
 'Slider Station',
 '101 Beer Kitchen',
 "Uncle Bill's Pet Centers Express",
 "Gigi's Cupcakes of Indianapolis",
 'Korave',
 "Ralston's Drafthouse",
 'The Eagle',
 "Abbott's Also Candy Store",
 'Sub Zero Nitrogen Ice Cream',
 'Cornerstone Coffee House',
 'Vitality Bowls',
 'Some Guys Pizza Pasta Grill',
 'Reis-Nichols Jewelers',
 'Lincoln Square Pancake Hou